---
# Question 2

In [886]:
import pandas as pd

# Initializing the data as a pandas DataFrame
manu_data = {
    'Step': ['a', 'b', 'c', 'd', 'e'],
    'Station': [1, 2, 3, 2, 1],
    'te': [0.16, 0.13, 0.13, 0.18, 0.14],
    'C_e^2': [0.5, 1.2, 0.8, 1.1, 0.6]
}

process_metrics = pd.DataFrame(manu_data)

# Display the DataFrame
print(process_metrics)


  Step  Station    te  C_e^2
0    a        1  0.16    0.5
1    b        2  0.13    1.2
2    c        3  0.13    0.8
3    d        2  0.18    1.1
4    e        1  0.14    0.6


In [887]:
process_metrics['E[S]^2'] = process_metrics['te'] ** 2
process_metrics['var[S]'] = process_metrics['C_e^2'] * process_metrics['te'] ** 2
process_metrics['E[S^2]'] = process_metrics['te'] ** 2 + process_metrics['var[S]']
# Display the DataFrame
print(process_metrics)

  Step  Station    te  C_e^2  E[S]^2   var[S]   E[S^2]
0    a        1  0.16    0.5  0.0256  0.01280  0.03840
1    b        2  0.13    1.2  0.0169  0.02028  0.03718
2    c        3  0.13    0.8  0.0169  0.01352  0.03042
3    d        2  0.18    1.1  0.0324  0.03564  0.06804
4    e        1  0.14    0.6  0.0196  0.01176  0.03136


Notice in this question now we will play with multiple machines in the system

In [888]:
import numpy as np

# Define the coefficient matrix A and the constants vector b
A = np.array([
    #A      B       C       D       E
    [1,     0,      -1/10,  0,      0],
    [-1,    1,      0,      0,      0],
    [0,     -1,     1,      0,      0],
    [0,     0,      -9/10,  1,      0],
    [0,     0,      0,      -1,     1]
])

# Right-hand side constants vector (all zero in this case)
b = np.array([5, 0, 0, 0, 0])

# Solve the system using np.linalg.solve if it’s square and has a unique solution
# If it does not have a unique solution, we can use np.linalg.lstsq to get a least-squares solution
try:
    lam = np.linalg.solve(A, b)
except np.linalg.LinAlgError:
    # If matrix A is singular, use least-squares solution
    lam, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

# The flow rates are the following 
lam


array([5.55555556, 5.55555556, 5.55555556, 5.        , 5.        ])

In [889]:
# Define the lambda inflow rates for each step
lambda_inflow = [lam[0], lam[1], lam[2], lam[3], lam[4]]

# Add the lambda inflow rates to the station_metrics dataframe
process_metrics['lambda_inflow'] = lambda_inflow

# Display the updated DataFrame
print(process_metrics)

  Step  Station    te  C_e^2  E[S]^2   var[S]   E[S^2]  lambda_inflow
0    a        1  0.16    0.5  0.0256  0.01280  0.03840       5.555556
1    b        2  0.13    1.2  0.0169  0.02028  0.03718       5.555556
2    c        3  0.13    0.8  0.0169  0.01352  0.03042       5.555556
3    d        2  0.18    1.1  0.0324  0.03564  0.06804       5.000000
4    e        1  0.14    0.6  0.0196  0.01176  0.03136       5.000000


Let's articulate this as the individual workstation flow rates

In [890]:
lam1=lam[1-1]+lam[5-1]
lam1

10.555555555555555

In [891]:
lam2=lam[2-1]+lam[4-1]
lam2

10.555555555555555

In [892]:
lam3=lam[3-1]
lam3

5.555555555555555

In [893]:
# Create a new DataFrame with 'Station' and 'lambda_inflow' columns using lam1, lam2, and lam3
station_metrics = pd.DataFrame({
    'Station': [1, 2, 3],
    'lambda_inflow': [lam1, lam2, lam3]
})

# Display the new DataFrame
print(station_metrics)

   Station  lambda_inflow
0        1      10.555556
1        2      10.555556
2        3       5.555556


---

Now let's get the utilization

In [894]:
station_metrics['te'] = [lam[0]/(lam[0]+lam[4])*process_metrics['te'][0]+lam[4]/(lam[0]+lam[4])*process_metrics['te'][4],
                         lam[1]/(lam[1]+lam[3])*process_metrics['te'][1]+lam[3]/(lam[1]+lam[3])*process_metrics['te'][3],
                         process_metrics['te'][2]]

# Display the updated DataFrame
print(station_metrics)

   Station  lambda_inflow        te
0        1      10.555556  0.150526
1        2      10.555556  0.153684
2        3       5.555556  0.130000


In [895]:

station_metrics['E[S]^2'] = station_metrics['te'] ** 2

# Display the updated DataFrame
print(station_metrics)

   Station  lambda_inflow        te    E[S]^2
0        1      10.555556  0.150526  0.022658
1        2      10.555556  0.153684  0.023619
2        3       5.555556  0.130000  0.016900


In [896]:

station_metrics['E[S^2]'] = [lam[0]/(lam[0]+lam[4])*process_metrics['E[S^2]'][0]+lam[4]/(lam[0]+lam[4])*process_metrics['E[S^2]'][4],
                             lam[1]/(lam[1]+lam[3])*process_metrics['E[S^2]'][1]+lam[3]/(lam[1]+lam[3])*process_metrics['E[S^2]'][3],
                             process_metrics['E[S^2]'][2]]
# Display the updated DataFrame
print(station_metrics)

   Station  lambda_inflow        te    E[S]^2    E[S^2]
0        1      10.555556  0.150526  0.022658  0.035065
1        2      10.555556  0.153684  0.023619  0.051798
2        3       5.555556  0.130000  0.016900  0.030420


In [897]:
station_metrics['var(S)'] = station_metrics['E[S^2]'] - station_metrics['E[S]^2']
# Display the updated DataFrame
print(station_metrics)

   Station  lambda_inflow        te    E[S]^2    E[S^2]    var(S)
0        1      10.555556  0.150526  0.022658  0.035065  0.012407
1        2      10.555556  0.153684  0.023619  0.051798  0.028179
2        3       5.555556  0.130000  0.016900  0.030420  0.013520


In [898]:
station_metrics['Ce^2'] = station_metrics['var(S)'] / station_metrics['te'] ** 2
# Display the updated DataFrame
print(station_metrics)

   Station  lambda_inflow        te    E[S]^2    E[S^2]    var(S)      Ce^2
0        1      10.555556  0.150526  0.022658  0.035065  0.012407  0.547577
1        2      10.555556  0.153684  0.023619  0.051798  0.028179  1.193076
2        3       5.555556  0.130000  0.016900  0.030420  0.013520  0.800000


In [899]:
station_metrics['m'] = [2, 2, 1]
# Display the updated DataFrame
print(station_metrics)

   Station  lambda_inflow        te    E[S]^2    E[S^2]    var(S)      Ce^2  m
0        1      10.555556  0.150526  0.022658  0.035065  0.012407  0.547577  2
1        2      10.555556  0.153684  0.023619  0.051798  0.028179  1.193076  2
2        3       5.555556  0.130000  0.016900  0.030420  0.013520  0.800000  1


In [900]:
# Calculate te^2 for each station
station_metrics['u'] = station_metrics['lambda_inflow']*station_metrics['te'] / station_metrics['m'] 
# Display the updated DataFrame
print(station_metrics)

   Station  lambda_inflow        te    E[S]^2    E[S^2]    var(S)      Ce^2  \
0        1      10.555556  0.150526  0.022658  0.035065  0.012407  0.547577   
1        2      10.555556  0.153684  0.023619  0.051798  0.028179  1.193076   
2        3       5.555556  0.130000  0.016900  0.030420  0.013520  0.800000   

   m         u  
0  2  0.794444  
1  2  0.811111  
2  1  0.722222  


---

Now let's get the Variance of the combined distributions for the purpose of calculating the SCV for each station's service time.

---

Now let's find the CSV of the arrivals

In [901]:
p12=lam[0]/(lam[0]+lam[4])
p12

0.5263157894736842

In [902]:
p21=lam[4]/(lam[4]+lam[1])
p21

0.4736842105263158

In [903]:
p31=.1
p32=.9

In [904]:
p23=lam[1]/(lam[1]+lam[4])
p23

0.5263157894736842

In [905]:
from sympy import symbols, Eq, solve

# Define symbols for the variables
scv_a_1, scv_a_2, scv_a_3 = symbols('scv_a_1, scv_a_2, scv_a_3')

$$C_a^2(1)$$

In [906]:
p21=p21
scv_e_2=2.087

p31=0.1
scv_e_3=1

m2=m[2-1]
m3=m[3-1]

eq1 = Eq(scv_a_1, 5/lam1
         +lam2*p21/lam1*(p21*(1+(1-station_metrics['u'][2-1]**2)*(scv_a_2-1)+(station_metrics['u'][2-1]**2)*(scv_e_2-1)/(m2**(1/2)))+1-p21)
         +lam3*p31/lam1*(p31*(1+(1-station_metrics['u'][3-1]**2)*(scv_a_3-1)+(station_metrics['u'][3-1]**2)*(scv_e_3-1)/(m3**(1/2)))+1-p31)
         )

$$ C_a^2(2)$$

In [907]:
p12=p12
scv_e_2=2.087

p32=0.9
scv_e_3=1

m2=m[2-1]
m3=m[3-1]

eq2 = Eq(scv_a_2,
         lam1*p12/lam2*(p12*(1+(1-station_metrics['u'][2-1]**2)*(scv_a_2-1)+(station_metrics['u'][2-1]**2)*(scv_e_2-1)/(m2**(1/2)))+1-p12)
         +lam3*p32/lam2*(p32*(1+(1-station_metrics['u'][3-1]**2)*(scv_a_3-1)+(station_metrics['u'][3-1]**2)*(scv_e_3-1)/(m3**(1/2)))+1-p32)
)

$$ C_a^2(3)$$

In [908]:
p23=p23
scv_e_3=1

eq3 = Eq(scv_a_3,
       lam2*p23/lam3*(p23*((1-station_metrics['u'][3-1]**2)*(scv_a_3)+(station_metrics['u'][3-1]**2)*(scv_e_3)))+1-p23)

In [909]:
# Solve the system of equations
solution = solve((eq1, eq2, eq3), (scv_a_1, scv_a_2, scv_a_3))

print("Solution:", solution)
scv_a_1 = solution[scv_a_1]
scv_a_2 = solution[scv_a_2]
scv_a_3 = solution[scv_a_3]

Solution: {scv_a_1: 1.12534047727114, scv_a_2: 1.15474132996438, scv_a_3: 1.00000000000000}


---

WIP, CT, and TH of System

In [ ]:
scv_e_1=0.1505213
m1=2
CT1=(scv_a_1+scv_e_1)/2*(station_metrics['u'][1-1]**((2*(m1+1))**(1/2)-1))/((1-station_metrics['u'][1-1])*m1)*station_metrics['te'][1-1]+station_metrics['te'][1-1]
CT1

0.317854833554569

In [911]:
CT2=(scv_a_2+scv_e_2)/2*(u2**((2*(m2+1))**(1/2)-1))/((1-u2)*m2)*te2+te2
CT2

0.640488174671278

In [912]:
CT3=(scv_a_3+scv_e_3)/2*(u3**((2*(m3+1))**(1/2)-1))/((1-u3)*m3)*te3+te3
CT3

0.468000000000000

In [913]:
WIP1=lam1*CT1
WIP1

3.35513435418712

In [914]:
WIP2=lam2*CT2
WIP2

6.76070851041904

In [915]:
WIP3=lam3*CT3
WIP3

2.60000000000000

In [916]:
WIP=WIP1+WIP2+WIP3
WIP

12.7158428646062

In [917]:
CT=WIP/5
CT

2.54316857292123